<a href="https://colab.research.google.com/github/KanthiKiranS/KK-Repo/blob/master/Pheme_Veracity_5_75.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import fastai

In [2]:
from fastai.text import * 

In [3]:
def read_veracity(path_to_file, shuffle=False):
    def enc(col):
        if col['label'] == 2.0:
            return 'Unverified'
        elif col['label'] == 1.0:
            return 'True'
        elif col['label'] == 0.0:
            return 'False'

    df = pd.read_csv(path_to_file)
    df = df.drop(['Unnamed: 0', '0'], 1)
    df.columns = ['text', 'label']
    df['label'] = df.apply(enc, 1)
    if shuffle:
        df = df.sample(frac=1).reset_index(drop=True)

    return df.dropna().iloc[:, ::-1]

In [4]:
# For veracity
train = read_veracity('modifiedTrain_New.csv', True)
test = read_veracity('modifiedTest_New.csv')

In [5]:
train.to_csv('train.csv')
test.to_csv('test.csv')

In [6]:
pd.concat([train['text'], test['text']]).reset_index(drop=True).to_csv('text.csv')

In [7]:
data_lm = TextLMDataBunch.from_csv('', 'text.csv')
data_clas = TextClasDataBunch.from_csv('', 'train.csv', vocab=data_lm.train_ds.vocab, text_cols='text', label_cols='label', bs=32)

In [8]:
data_lm.save('data_lm_export.pkl')
data_clas.save('data_clas_export.pkl')

In [9]:
data_lm = load_data('', 'data_lm_export.pkl')
data_clas = load_data('', 'data_clas_export.pkl', bs=16)

In [10]:
learn = language_model_learner(data_lm, AWD_LSTM, drop_mult=0.5)
learn.fit_one_cycle(1, 1e-2, moms=(0.8, 0.7))

epoch,train_loss,valid_loss,accuracy,time
0,4.581627,4.256894,0.292742,01:04


In [11]:
learn.unfreeze()
learn.fit_one_cycle(1, 1e-3, moms=(0.8, 0.7))

epoch,train_loss,valid_loss,accuracy,time
0,4.060358,3.953680,0.329262,01:15


In [12]:
learn.predict("This is a review about", n_words=10)

'This is a review about Treason . Sorry , so e Islam'

In [13]:
learn.save_encoder('ft_enc')

In [14]:
learn = text_classifier_learner(data_clas, AWD_LSTM, drop_mult=0.5)
learn.load_encoder('ft_enc')

RNNLearner(data=TextClasDataBunch;

Train: LabelList (50430 items)
x: TextList
xxbos xxmaj no , murderers often use religion as an excuse to murder . @spcjackalope @politibunny @shidelerk @skiplacombe,xxbos xxmaj remember : do xxup not tweet location of police in # xxmaj ottawa # ottawashooting,xxbos @onthemedia xxunk,xxbos @lisarobertser @australian xxmaj so true ! xxmaj what next ? xxmaj no country is safe from this .,xxbos xxunk @iowahawkblog i give hotheads no credit whatsoever . xxmaj what i know is that we know almost nothing about what happened . # humble
y: CategoryList
False,False,False,True,False
Path: .;

Valid: LabelList (12608 items)
x: TextList
xxbos @once_written @occupylondon xxmaj you and me both my friend , when will the consensus of mankind learn the monetary system is slavery .,xxbos @cousindangereux @stillgray the kicker was they actually lied about the blood,xxbos @pornpops69 so you 're okay making shit up about other religions but when someone quotes yours they '

In [15]:
data_clas.show_batch()

text,target
xxbos “ @wesleylowery : a few feet from where # michaelbrown was killed : # xxmaj ferguson http : / / t.co / xxunk ” \n “ @wesleylowery : a few feet from where # michaelbrown was killed : # xxmaj ferguson http : / / t.co / xxunk \n “ @wesleylowery : a few feet from where # michaelbrown was killed : # xxmaj ferguson http :,False
"xxbos “ @theepharoah : xxup please xxup dont xxup try xxup to xxup buy xxup me xxup out , xxup money xxup or xxup anything . xxup my xxup story xxup is xxup for xxup justice . # xxmaj ferguson http : / / t.co / xxunk ” @goldietaylor",False
"xxbos @nbcnews xxup hi xxup there . xxup it xxup 's xxup suziq . xxup americans xxup had xxup better xxup be xxup on xxup overtime "" xxup keeping xxup their xxup guards xxup xxunk xxup xxunk : xxup from xxup now xxup xxunk xxup suziq",True
xxbos “ @liztilley84 : xxmaj flag in window of xxmaj sydney xxmaj lindt cafe not an xxup isis flag . xxmaj reads : ‘ xxmaj there is no xxmaj god but xxmaj allah and xxmaj muhammad is the messenger of xxmaj god ’,True
"xxbos @toolieo1 "" xxmaj only 1 xxmaj god , and xxmaj mohammed is his prophet . "" xxmaj do n't need to know xxmaj arabic to recognize it from the xxmaj al xxmaj qaeda , xxmaj taliban , and xxmaj saudi flags .",False


In [16]:
learn.fit_one_cycle(1, 1e-2, moms=(0.8, 0.7))

epoch,train_loss,valid_loss,accuracy,time
0,0.600824,0.553257,0.719305,00:58


In [17]:
learn.freeze_to(-2)
learn.fit_one_cycle(1, slice(5e-3/2., 5e-3), moms=(0.8, 0.7))

epoch,train_loss,valid_loss,accuracy,time
0,0.540737,0.466085,0.775539,01:08


In [18]:
learn.unfreeze()
learn.fit_one_cycle(10, slice(2e-3/100, 2e-3), moms=(0.8, 0.7))

epoch,train_loss,valid_loss,accuracy,time
0,0.511062,0.443047,0.790371,02:02
1,0.449492,0.402850,0.818607,02:00
2,0.415632,0.345610,0.842402,01:52
3,0.352385,0.314391,0.859692,01:57
4,0.314181,0.277672,0.875476,01:50
5,0.288533,0.277197,0.874683,01:51
6,0.197925,0.273561,0.880631,01:54
7,0.204923,0.268903,0.883169,01:54
8,0.180516,0.270260,0.884121,02:00
9,0.162069,0.278653,0.881980,01:53


In [19]:
l = []
for text in test['text']:
    l.append(str(learn.predict(text)[0]))

In [20]:
from sklearn.metrics import accuracy_score, f1_score

In [21]:
accuracy_score(test['label'], l)

0.7501306574683809

In [22]:
f1_score(test['label'], l, average='macro')

0.6391873961417422

In [23]:
learn.export('veracity.pkl')